In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from scipy import stats
import itertools
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
import plotTimeseries
import os
import pandas as pd
from pyspark.sql import Window
import matplotlib.ticker as ticker
from matplotlib.lines import Line2D
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch, Rectangle, Circle
from wordcloud import WordCloud


sns.set(font_scale=1.5)
sns.set_style('ticks')
%load_ext autoreload
%autoreload 2

#spark = SparkSession.builder.config("spark.driver.bindAddress","192.168.1.196").getOrCreate()
spark = SparkSession.builder.getOrCreate()

Extras

In [ ]:
# year difference for evidence with several timestamps
spark.read.parquet("../results/23.06/evidenceDated").select("publicationYear","curationYear","datasourceId").filter(F.col("publicationYear").isNotNull()&F.col("curationYear").isNotNull()).withColumn("window",F.col("curationYear")-F.col("publicationYear")).toPandas().groupby("datasourceId").median()

Novelty peaks overlap across resource

In [ ]:
year = 2005
resultsReal,resultsShuffled = [],[]
data = (spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/noveltyCorrelation/evidenceLink=direct/pairs")       
        .filter((F.col("realYearA")>=year)&(F.col("realYearB")>=year)).toPandas())
for s1, s2 in itertools.product(data.datasourceIdA.unique(), data.datasourceIdA.unique()):
    # real
    x=data[(data.datasourceIdA==s1)&(data.datasourceIdB==s2)][["realYearA","realYearB"]].dropna().realYearA
    y=data[(data.datasourceIdA==s1)&(data.datasourceIdB==s2)][["realYearA","realYearB"]].dropna().realYearB
    try:
        rvalue, pvalue = stats.pearsonr(x, y)
        resultsReal.append([s1, s2,round(rvalue,2), round(pvalue,3), len(x)])
    except ValueError:
        resultsReal.append([s1, s2, None, None, len(x)])
    # shuffled
    x=data[(data.datasourceIdA==s1)&(data.datasourceIdB==s2)][["shuffledYearA","shuffledYearB"]].dropna().shuffledYearA
    y=data[(data.datasourceIdA==s1)&(data.datasourceIdB==s2)][["shuffledYearA","shuffledYearB"]].dropna().shuffledYearB
    try:
        rvalue, pvalue = stats.pearsonr(x, y)
        resultsShuffled.append([s1, s2,round(rvalue,2), round(pvalue,3), len(x)])
    except ValueError:
        resultsShuffled.append([s1, s2, None, None, len(x)])
resultsReal = pd.DataFrame(resultsReal, columns=["datasourceIdA","datasourceIdB","r","p","n"])
resultsShuffled = pd.DataFrame(resultsShuffled, columns=["datasourceIdA","datasourceIdB","r","p","n"])
resultsReal.to_csv("../results/23.06/correlation{}.csv".format(year), index=False)
resultsShuffled.to_csv("../results/23.06/correlationShuffled{}.csv".format(year), index=False)

In [ ]:
# correlation clustermap
year = 2005
data = pd.read_csv("../results/23.06/correlation{}.csv".format(year)).copy()
data = data[(data["p"]<=0.05)]
data = (data
        .pivot(index="datasourceIdA", columns="datasourceIdB", values="r")
        .merge(plotTimeseries.getDatatypeForDatasource(output="dataframe").rename(columns={"datasourceId":"datasourceIdA"}), on="datasourceIdA", how="inner")
        .replace(plotTimeseries.getDatasourceToName(output="dict"))
        .rename(columns=plotTimeseries.getDatasourceToName(output="dict"))
        .set_index("datasourceIdA")
        .replace(plotTimeseries.getDatatypeToName(output="dict"))
        .rename(columns={"datatypeId":"Category"})
)
categories = data.pop("Category")
row_colors = categories.map(plotTimeseries.getDatatypeToColour(by="label"))
plt.figure(figsize=(15,15))
sns.set(font_scale=1.5, style="ticks")
ax = sns.clustermap(data.fillna(0), lw=1, cmap="Blues", square=True, vmin=0, vmax=1, row_colors=row_colors, cbar_pos=(0, .35, .03, .4), cbar_kws={"label":"Pearson correlation coefficient"})
ax.ax_row_dendrogram.set_visible(False)
ax.ax_col_dendrogram.set_visible(False)
ax.ax_heatmap.set_xlabel("")
ax.ax_heatmap.set_ylabel("")
plt.savefig("../results/23.06/plots/correlations.png", dpi=300, bbox_inches="tight")

In [ ]:
# correlation heatmap annotated
data = pd.read_csv("../results/23.06/correlation{}.csv".format(year)).copy()
data["annot"] = data.apply(lambda row: "r={}\np={}\nN={}".format(row.r, row.p, row.n), axis=1)
plt.figure(figsize=(30,30))
ax = sns.heatmap(data.pivot(index="datasourceIdA", columns="datasourceIdB", values="r"), 
            annot=data.pivot(index="datasourceIdA", columns="datasourceIdB", values="annot").fillna(""), fmt="",
            lw=1, cbar=False, cmap="Blues", vmin=0, vmax=1)
ax.set_xlabel("")
ax.set_ylabel("")
plt.savefig("../results/23.06/plots/correlations_annot.png", dpi=300)

In [ ]:
# correlation heatmap for shuffled peaks annotated
data = pd.read_csv("../results/23.06/correlationShuffled{}.csv".format(year)).copy()
data["annot"] = data.apply(lambda row: "r={}\np={}\nN={}".format(row.r, row.p, row.n), axis=1)
plt.figure(figsize=(30,30))
ax = sns.heatmap(data.pivot(index="datasourceIdA", columns="datasourceIdB", values="r"), 
            annot=data.pivot(index="datasourceIdA", columns="datasourceIdB", values="annot").fillna(""), fmt="",
            lw=1, cbar=False, cmap="Blues", vmin=0, vmax=1)
ax.set_xlabel("")
ax.set_ylabel("")
plt.savefig("../results/23.06/plots/correlationsShuffled_annot.png", dpi=300)

Bell plots

In [35]:
yearsRange = "2010-2012"
data = pd.read_csv("/Users/mariaf/TargetEngine/results/23.06/noveltyBenchmark/bells/years="+yearsRange).replace(plotTimeseries.getDatatypeToName(output="dict"))
data["years"] = data.apply(lambda row: row.years.replace("sum(","").replace("-","−").replace(",",", ")[:-1], axis=1)
data = data[(data.datatypeId!="Pre-clinical phase")&(data.datatypeId!="Known drug")]
datatypes = ["Human genetic data", "Literature", "Animal model", "Affected pathway", "RNA expression","Clinical phase I/II","Clinical phase III","Clinical phase IV"][::-1]

In [ ]:
plt.figure()
fig, ax = plt.subplots(figsize=(6,6))
sns.set(font_scale=1.5, style="ticks")
for datatypeId in datatypes:
    sns.lineplot(data=data[data.datatypeId==datatypeId], x="years", y="%withPeak", color="w", lw=10, legend=False, ax=ax, alpha=1)
    sns.lineplot(data=data[(data.datatypeId==datatypeId)], x="years", y="%withPeak", color=plotTimeseries.getDatatypeToColour(by="label")[datatypeId], lw=7, legend=False, ax=ax, alpha=1, label=datatypeId)        

ax.set_ylabel("% of targets with novelty peak")
ax.set_xlabel("Peak year − Approval year")
ax.set_ylim(-3,100)
ax.grid(axis="y", lw=.5)
ax.set_title("{} novel drug targets".format(yearsRange), fontweight="bold")
ax.set_xticklabels(labels=ax.get_xticklabels(), rotation=40, ha="right")
ax.axvline(x=6, color="k", linestyle="--", lw=1)
sns.despine()
plt.savefig("../results/23.06/plots/bells/{}.png".format(yearsRange), dpi=300, bbox_inches="tight")

Cascade plots

In [ ]:
data = spark.read.parquet("/Users/mariaf/TargetEngine/results/23.06/noveltyBenchmark/cascadesBoxplot").orderBy("years").select("datatypeId","window","years").toPandas()
for datatypeId in data.datatypeId.unique():
    sns.set(font_scale=1.7, style="ticks")
    fig, ax = plt.subplots(figsize=(5,5))
    sns.boxplot(data=data[data.datatypeId==datatypeId], x="years", y="window", zorder=1, color=plotTimeseries.getDatatypeToColour()[datatypeId], fliersize=0, boxprops={"edgecolor":"w"}, medianprops={"color":"w"})
    
    ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right",va="top")
    ax.set_xlabel("Drug approval year")
    ax.set_ylabel("Peak year − Approval year")
    ax.set_title(plotTimeseries.getDatatypeToName(output="dict")[datatypeId]+" peak")
    ax.set_ylim(25,-25)
    ax.grid(axis="y", lw=.5, zorder=0)
    ax.axhline(y=0, lw=1, linestyle="--", color="k", zorder=2)
    sns.despine()
    plt.savefig("../results/23.06/plots/cascades/{}.png".format(datatypeId.replace("/","-")), dpi=300, bbox_inches="tight")

Novel drug targets

In [ ]:
(spark.read.option("header", "true").csv("/Users/mariaf/ChEMBL/CHEMBL_34/drugs.csv")
  .join(spark.read.parquet("/Users/mariaf/OT_platform/23.06/evidence/sourceId=chembl").filter(F.col("clinicalPhase")==4).select("drugId","targetId","diseaseId").distinct(), "drugId", "left")
  .filter(F.col("targetId").isNotNull()&(F.col("firstApprovalYear").isNotNull()))
  .withColumn("minApprovalYear", F.min("firstApprovalYear").over(Window.partitionBy("targetId")))
  .filter(F.col("minApprovalYear")==F.col("firstApprovalYear"))
  .select("drugId","drug","targetId","diseaseId",F.col("firstApprovalYear").alias("approvalYear")).distinct()
).write.parquet("../data/novelDrugTargets/novelDrugTargets")

Timestamps

In [ ]:
data = (spark.read.parquet("../results/23.06/reportEvidenceDirectDated/")
        .join(spark.createDataFrame(plotTimeseries.getDatasourceToName(), schema=["datasourceId", "datasourceName"]), "datasourceId", "left")
        .join(spark.createDataFrame(plotTimeseries.getDatatypeForDatasource(), schema=["datasourceId", "datatypeId"]), "datasourceId", "left")
        .join(spark.createDataFrame(plotTimeseries.getDatatypeToName(), schema=["datatypeId", "datatypeName"]), "datatypeId", "left")
        .groupby("datasourceName", "datatypeName").agg(F.sum("nAll").alias("Evidence"))
        .withColumnRenamed("datasourceName","Source")
).toPandas()
data["sortedDatasourceName"] = data.Source.astype("category")
data["sortedDatasourceName"] = data.sortedDatasourceName.cat.set_categories(plotTimeseries.datasourceOrder)
data = data.sort_values("sortedDatasourceName").drop("sortedDatasourceName", axis=1)
data.sum()

In [ ]:
plt.figure(figsize=(9, 6))
sns.set(font_scale=1.5)
sns.set_style('ticks')
plt.rcParams['legend.handlelength'] = 1.2
plt.rcParams['legend.handleheight'] = 1.51

data = (spark.read.parquet("../results/23.06/reportEvidenceDirectDated/")
        .filter((F.col("year")<=2023)).select("datasourceId","year")
        .join(spark.createDataFrame(plotTimeseries.getDatasourceToName(), schema=["datasourceId", "datasourceName"]), "datasourceId", "left")
        .join(spark.createDataFrame(plotTimeseries.getDatatypeForDatasource(), schema=["datasourceId", "datatypeId"]), "datasourceId", "left")
        .join(spark.createDataFrame(plotTimeseries.getDatatypeToName(), schema=["datatypeId", "datatypeName"]), "datatypeId", "left")
        .toPandas())
data["sortedDatasourceName"] = data.datasourceName.astype("category")
data["sortedDatasourceName"] = data.sortedDatasourceName.cat.set_categories(plotTimeseries.datasourceOrder)
data = data.sort_values("sortedDatasourceName").drop("sortedDatasourceName", axis=1)

ax = sns.boxplot(x="datasourceName", y="year", hue="datatypeName", data=data.replace({"Known drug":"Clinical"}), fliersize=0, width=.6, 
                 boxprops={"edgecolor":"w"}, medianprops={"color":"k"},
                 palette=plotTimeseries.getDatatypeToColour(by="label"))
ax.set_ylim(1950,2024.5)
ax.set_xlabel("")
ax.set_ylabel("Year of evidence timestamp")
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, ha='right')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(axis="y", lw=.5)
ax.legend(title="Category:", fancybox=False, frameon=True, facecolor="w", edgecolor="k", framealpha=1, fontsize=15, title_fontsize=15)
plt.savefig("../results/23.06/plots/timestampsBoxplot.png", dpi=300, bbox_inches="tight", transparent=True)

In [ ]:
data = (spark.read.parquet("../results/23.06/reportEvidenceDirectDated/")
        .join(spark.createDataFrame(plotTimeseries.getDatasourceToName(), schema=["datasourceId", "datasourceName"]), "datasourceId", "left")
        .groupby("datasourceName").agg(F.sum("nDated").alias("Dated evidence"))
        .withColumnRenamed("datasourceName","Source")
).toPandas()
data["sortedDatasourceName"] = data.Source.astype("category")
data["sortedDatasourceName"] = data.sortedDatasourceName.cat.set_categories(plotTimeseries.datasourceOrder)
data = data.sort_values("sortedDatasourceName").drop("sortedDatasourceName", axis=1)
data

Temporal profiles for case example

In [ ]:
# evidence
case = (spark.read.parquet("../results/23.06/evidenceIndirectDated/").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
        .join(spark.createDataFrame([["chembl","ChEMBL"],["ot_genetics_portal","OT Genetics"],["europepmc","Europe PMC"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
 .select("datasourceName","year","score")
).toPandas()

sns.set(font_scale=1.7, style="ticks")
fig, ax = plt.subplots(figsize=(5,5))
sns.scatterplot(data=case, x="year", y="score", hue="datasourceName", palette={"Europe PMC":plotTimeseries.getDatatypeToColour()["literature"],
                                                                               "OT Genetics":plotTimeseries.getDatatypeToColour()["genetic_association"],
                                                                               "ChEMBL":plotTimeseries.getDatatypeToColour()["known_drug"]}, hue_order=["Europe PMC","OT Genetics","ChEMBL"], s=500, alpha=.4, ax=ax)
ax.set_xlim(2000,2024)
ax.set_ylim(0,1.05)
ax.set_xlabel("")
ax.set_ylabel("Score")
ax.set_title("Evidence")
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
ax.legend(frameon=False, loc="upper left")
sns.despine()
plt.savefig("../results/23.06/plots/case/evidence.png", dpi=300, bbox_inches="tight")

In [ ]:
# score
sns.set(font_scale=1.7, style="ticks")
data = (spark.read.parquet("../results/23.06/associationByDatasourceIndirectOverYears/").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
.join(spark.createDataFrame([["chembl","ChEMBL"],["ot_genetics_portal","OT Genetics"],["europepmc","Europe PMC"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
.toPandas())

fig, ax = plt.subplots(figsize=(5,5))
sns.lineplot(data=data, x="year", y="score", hue="datasourceName", palette={"Europe PMC":plotTimeseries.getDatatypeToColour()["literature"],
                                                                               "OT Genetics":plotTimeseries.getDatatypeToColour()["genetic_association"],
                                                                               "ChEMBL":plotTimeseries.getDatatypeToColour()["known_drug"]}, hue_order=["Europe PMC","OT Genetics","ChEMBL"], lw=3)
ax.set_xlim(2000,2023)
ax.set_ylim(0,1.05)
ax.set_xlabel("")
ax.set_ylabel("Score")
ax.set_title("Association")
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
plt.rcParams['legend.handlelength'] = 1.5
plt.rcParams['legend.handleheight'] = 1.51
ax.legend(frameon=False, loc="upper left")
sns.despine()
plt.savefig("../results/23.06/plots/case/score.png", dpi=300, bbox_inches="tight")

In [ ]:
# novelty
sns.set(font_scale=1.7, style="ticks")
data = (spark.read.parquet("../results/23.06/associationByDatasourceIndirectOverYears/").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
.join(spark.createDataFrame([["chembl","ChEMBL"],["ot_genetics_portal","OT Genetics"],["europepmc","Europe PMC"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
.toPandas())

fig, ax = plt.subplots(figsize=(5,5))
sns.lineplot(data=data, x="year", y="novelty", hue="datasourceName", palette={"Europe PMC":plotTimeseries.getDatatypeToColour()["literature"],
                                                                               "OT Genetics":plotTimeseries.getDatatypeToColour()["genetic_association"],
                                                                               "ChEMBL":plotTimeseries.getDatatypeToColour()["known_drug"]},  hue_order=["Europe PMC","OT Genetics","ChEMBL"], lw=6)
ax.set_xlim(2000,2023)
ax.set_ylim(0,1.05)
ax.set_xlabel("")
ax.set_ylabel("Score")
ax.set_title("Novelty")
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
plt.rcParams['legend.handlelength'] = 1.5
plt.rcParams['legend.handleheight'] = 1.51
ax.legend(frameon=False, loc="upper left")
sns.despine()
plt.savefig("../results/23.06/plots/case/novelty.png", dpi=300, bbox_inches="tight")

In [ ]:
# evidence+novelty
evidence = (spark.read.parquet("../results/23.06/evidenceIndirectDated/").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
        .join(spark.createDataFrame([["chembl","ChEMBL evidence"],["ot_genetics_portal","OT Genetics evidence"],["europepmc","Europe PMC evidence"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
 .select("datasourceName","year","score")
).toPandas()
novelty = (spark.read.parquet("../results/23.06/associationByDatasourceIndirectOverYears/").filter((F.col("diseaseId")=="MONDO_0004979")&(F.col("targetId")=="ENSG00000145777"))
.join(spark.createDataFrame([["chembl","ChEMBL novelty"],["ot_genetics_portal","OT Genetics novelty"],["europepmc","Europe PMC novelty"]], schema=["datasourceId","datasourceName"]), "datasourceId","left")
.toPandas())

sns.set(font_scale=1.7, style="ticks")
fig, ax = plt.subplots(figsize=(5,5))
sns.scatterplot(data=evidence, x="year", y="score", hue="datasourceName", palette={"Europe PMC evidence":plotTimeseries.getDatatypeToColour()["literature"],
                                                                               "OT Genetics evidence":plotTimeseries.getDatatypeToColour()["genetic_association"],
                                                                               "ChEMBL evidence":plotTimeseries.getDatatypeToColour()["known_drug"]}, hue_order=["Europe PMC evidence","OT Genetics evidence","ChEMBL evidence"], s=500, alpha=.4, ax=ax)
sns.lineplot(data=novelty, x="year", y="novelty", hue="datasourceName", palette={"Europe PMC novelty":"w",
                                                                               "OT Genetics novelty":"w",
                                                                               "ChEMBL novelty":"w"}, lw=10, legend=False)
sns.lineplot(data=novelty, x="year", y="novelty", hue="datasourceName", palette={"Europe PMC novelty":plotTimeseries.getDatatypeToColour()["literature"],
                                                                               "OT Genetics novelty":plotTimeseries.getDatatypeToColour()["genetic_association"],
                                                                               "ChEMBL novelty":plotTimeseries.getDatatypeToColour()["known_drug"]},  hue_order=["Europe PMC novelty","OT Genetics novelty","ChEMBL novelty"], lw=6)

ax.set_xlim(2000,2024)
ax.set_ylim(0,1.05)
ax.set_xlabel("")
ax.set_ylabel("Score")
ax.set_title("TSLP + asthma")
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
ax.legend(frameon=False, loc='center left', bbox_to_anchor=(1, 0.5))
sns.despine()
plt.savefig("../results/23.06/plots/case/evidence+novelty.png", dpi=300, bbox_inches="tight")

GE for approvals in Rusina 2023 from OT platform 23.02

In [ ]:
approvalsRusinaGE = pd.read_excel("/Users/mariaf/TargetEngine/data/approvals/26125730.xlsx")#[["drugId","therapeuticArea","yearApproval","hasAnyGE","targetType","diseaseType","brandDrugName"]]


In [ ]:
pd.read_excel("/Users/mariaf/TargetEngine/data/approvals/26125730.xlsx")[["drugId","brandDrugName","therapeuticArea","yearApproval"]].to_csv("../data/approvals.tsv", index=False, sep="\t")
(spark.read.option("header","true").option("sep","\t").csv("../data/approvals.tsv") # Rusina Sup. Table
  .withColumnRenamed("drugId", "drugIds")
  .withColumn("drugId", F.explode_outer(F.split("drugIds", ", ")))
  .join(spark.read.parquet("/Users/mariaf/OT_platform/23.02/evidence/sourceId=chembl").select("diseaseId","targetId","drugId"), ["drugId"], "left")
  .groupby("drugIds","brandDrugName","therapeuticArea","yearApproval").agg(F.collect_set("diseaseId").alias("diseaseIds"), F.collect_set("targetId").alias("targetIds"))
  .withColumnRenamed("drugIds", "drugId")
 ).toPandas().to_csv("approvalsAutomaticPair.tsv", sep="\t", index=False)

Automatic GE extraction from OT_platform 23.02

In [ ]:
approvalsAutomaticGE = (
    # phase IV -> direct target-disease associations since 2013
    spark.read.parquet("/Users/mariaf/OT_platform/23.02/evidence/sourceId=chembl").filter(F.col("clinicalPhase")==4)
    .select("drugId", "diseaseId", "targetId", F.split("studyStartDate", "-").getItem(0).alias("studyStartYear"))
    .filter(F.col("studyStartYear")>=2013)
    # indirect GE
    .join(spark.read.parquet("/Users/mariaf/OT_platform/23.02/evidence/").select("datasourceId","diseaseId","targetId")
          .filter(~F.col("datasourceId").isin(["chembl","europepmc"])), 
          ["diseaseId","targetId"], "left")
)
print("drugId: ", approvalsAutomaticGE.select("drugId").distinct().count())
print("drugId with GE: ", approvalsAutomaticGE.filter(F.col("datasourceId").isNotNull()).select("drugId").distinct().count())
print("% drugId with GE: ", approvalsAutomaticGE.filter(F.col("datasourceId").isNotNull()).select("drugId").distinct().count()/approvalsAutomaticGE.select("drugId").distinct().count())

print("targetId: ", approvalsAutomaticGE.select("targetId").distinct().count())
print("targetId with GE: ", approvalsAutomaticGE.filter(F.col("datasourceId").isNotNull()).select("targetId").distinct().count())
print("% targetId with GE: ", approvalsAutomaticGE.filter(F.col("datasourceId").isNotNull()).select("targetId").distinct().count()/approvalsAutomaticGE.select("targetId").distinct().count())

print("targetId-diseaseId: ", approvalsAutomaticGE.select("targetId","diseaseId").distinct().count())
print("targetId-diseaseId with GE: ", approvalsAutomaticGE.filter(F.col("datasourceId").isNotNull()).select("targetId","diseaseId").distinct().count())
print("% targetId-diseaseId with GE: ", approvalsAutomaticGE.filter(F.col("datasourceId").isNotNull()).select("targetId","diseaseId").distinct().count()/approvalsAutomaticGE.select("targetId","diseaseId").distinct().count())

approvalsAutomaticGE = approvalsAutomaticGE.select("drugId","studyStartYear","datasourceId").distinct().toPandas()
approvalsAutomaticGE["studyStartYear"] = approvalsAutomaticGE.apply(lambda x: int(x.studyStartYear), axis=1)

Rusina section

In [ ]:
approvalsRusinaIndirectGE = pd.read_excel("/Users/mariaf/TargetEngine/data/approvals/26125730.xlsx")[["drugId","therapeuticArea","yearApproval","hasAnyGE","targetType","diseaseType"]]
approvalsRusinaDirectGE = pd.read_excel("/Users/mariaf/TargetEngine/data/approvals/26125730.xlsx")[["drugId","therapeuticArea","yearApproval","hasAnyGE","targetType","diseaseType"]]
approvalsRusinaGE = pd.read_excel("/Users/mariaf/TargetEngine/data/approvals/26125730.xlsx")[["drugId","therapeuticArea","yearApproval","hasAnyGE","targetType","diseaseType"]]
approvalsRusinaGE["therapeuticArea"] = approvalsRusinaGE.apply(lambda row: "oncology" if row.therapeuticArea == "Oncology" else "non-oncology", axis=1)

results = (
    # automatic GE extraction from platform 23.02    
    #approvalsAutomaticGE.groupby("studyStartYear").count().drugId.reset_index().rename(columns={"drugId":"automaticDrugId","studyStartYear":"year"})
    #.merge(approvalsAutomaticGE[approvalsAutomaticGE.datasourceId.notnull()].groupby("studyStartYear").count().drugId.reset_index().rename(columns={"drugId":"automaticGE", "studyStartYear":"year"}), on="year", how="left")
    
    # Rusina approvals
    approvalsRusinaIndirectGE.groupby(["yearApproval"]).count()[["drugId"]].reset_index().rename(columns={"drugId":"Approvals","yearApproval":"year"})
    
    # Rusina approvals supported by indirect GE linking original/interactor target and original/related disease
    .merge(approvalsRusinaGE[(approvalsRusinaGE.hasAnyGE==True)].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"Any indirect GE","yearApproval":"year"}), on="year", how="left")      
    # Rusina approvals supported by indirect GE linking original target and original disease
    .merge(approvalsRusinaIndirectGE[(approvalsRusinaIndirectGE.hasAnyGE==True)&(approvalsRusinaIndirectGE.targetType=="Original")&(approvalsRusinaIndirectGE.diseaseType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"%Indirect GE between original target and original disease","yearApproval":"year"}), on="year", how="left")
    # Rusina approvals supported by indirect GE linking original target and original/related disease
    .merge(approvalsRusinaIndirectGE[(approvalsRusinaIndirectGE.hasAnyGE==True)&(approvalsRusinaIndirectGE.targetType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"Indirect GE between original target and original/related disease","yearApproval":"year"}), on="year", how="left")
    # Rusina approvals supported by indirect GE linking original/interactor target and original disease
    .merge(approvalsRusinaIndirectGE[(approvalsRusinaIndirectGE.hasAnyGE==True)&(approvalsRusinaIndirectGE.diseaseType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"Indirect GE between original/interacting target and original disease","yearApproval":"year"}), on="year", how="left")    
    
    # Rusina approvals supported by direct GE linking original/interactor target and original/related disease
    .merge(approvalsRusinaGE[(approvalsRusinaDirectGE.hasAnyGE==True)].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"%Direct GE between original/interacting target and original/related disease","yearApproval":"year"}), on="year", how="left")      
    # Rusina approvals supported by direct GE linking original target and original disease
    .merge(approvalsRusinaIndirectGE[(approvalsRusinaDirectGE.hasAnyGE==True)&(approvalsRusinaDirectGE.targetType=="Original")&(approvalsRusinaDirectGE.diseaseType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"%Direct GE between original target and original disease","yearApproval":"year"}), on="year", how="left")
    # Rusina approvals supported by direct GE linking original target and original/related disease
    .merge(approvalsRusinaIndirectGE[(approvalsRusinaDirectGE.hasAnyGE==True)&(approvalsRusinaDirectGE.targetType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"Direct GE between original target and original/related disease","yearApproval":"year"}), on="year", how="left")
    # Rusina approvals supported by direct GE linking original/interactor target and original disease
    .merge(approvalsRusinaIndirectGE[(approvalsRusinaDirectGE.hasAnyGE==True)&(approvalsRusinaDirectGE.diseaseType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"Direct GE between original/interacting target and original disease","yearApproval":"year"}), on="year", how="left")    
)
#results["%automaticGE"] = results["automaticGE"]/results["automaticDrugId"]

results["%Any indirect GE"] = results["Indirect GE between original/interacting target and original/related disease"]/results["Approvals"]
# results["%Indirect GE between original target and original disease"] = results["Indirect GE between original target and original disease"]/results["Approvals"]
# results["%Indirect GE between original target and original/related disease"] = results["Indirect GE between original target and original/related disease"]/results["Approvals"]
# results["%Indirect GE between original/interacting target and original disease"] = results["Indirect GE between original/interacting target and original disease"]/results["Approvals"]

# results["%Direct GE between original/interacting target and original/related disease"] = results["Direct GE between original/interacting target and original/related disease"]/results["Approvals"]
# results["%Direct GE between original target and original disease"] = results["Direct GE between original target and original disease"]/results["Approvals"]
# results["%Direct GE between original target and original/related disease"] = results["Direct GE between original target and original/related disease"]/results["Approvals"]
# results["%Direct GE between original/interacting target and original disease"] = results["Direct GE between original/interacting target and original disease"]/results["Approvals"]


# ax = plt.figure().add_subplot()

# sns.lineplot(data=results, x="year", y="%Indirect GE between original/interacting target and original/related disease", marker=".", markersize=10, color="g", 
#              label="%Indirect GE between original/interacting target and original/related disease", ax=ax)
# sns.lineplot(data=results, x="year", y="Indirect GE between original/interacting target and original disease", marker=".", markersize=10, color="orange", ax=ax, 
#              label="Indirect GE between original/interacting target and original disease")
# sns.lineplot(data=results, x="year", y="Indirect GE between original target and original/related disease", marker=".", markersize=10, color="purple", ax=ax, 
#              label="%Indirect GE between original/interacting target and original disease")
# sns.lineplot(data=results, x="year", y="%Indirect GE between original target and original disease", marker=".", markersize=10, color="b", ax=ax, 
#              label="%Indirect GE between original target and original disease")

# sns.lineplot(data=results, x="year", y="%Direct GE between original/interacting target and original/related disease", marker=".", markersize=10, color="g", 
#              label="%Direct GE between original/interacting target and original/related disease", ax=ax, linestyle="--")
# sns.lineplot(data=results, x="year", y="Direct GE between original/interacting target and original disease", marker=".", markersize=10, color="orange", ax=ax, linestyle="--", 
#              label="Direct GE between original/interacting target and original disease")
# sns.lineplot(data=results, x="year", y="Direct GE between original target and original/related disease", marker=".", markersize=10, color="purple", ax=ax, linestyle="--", 
#              label="%Direct GE between original/interacting target and original disease")
# sns.lineplot(data=results, x="year", y="%Direct GE between original target and original disease", marker=".", markersize=10, color="b", ax=ax, linestyle="--", 
#              label="%Direct GE between original target and original disease")

# #sns.lineplot(data=results, x="year", y="%automaticGE", marker=".", markersize=10, color="r", label="Automatic GE", ax=ax)
# ax.set_ylim(0,1)
# ax.set_xlim(2013,2023)
# ax.set_ylabel("% Approvals")
# ax.set_xlabel("Approval year")
# leg = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), frameon=0)
# #plt.savefig("../results/23.02/plots/approvals.png", dpi=300, bbox_inches="tight")

In [ ]:
approvalsRusinaIndirectGE = pd.read_excel("/Users/mariaf/TargetEngine/data/approvals/26125730.xlsx")[["drugId","therapeuticArea","yearApproval","hasAnyGE","targetType","diseaseType"]]
approvalsRusinaDirectGE = pd.read_excel("/Users/mariaf/TargetEngine/data/approvals/26125730.xlsx")[["drugId","therapeuticArea","yearApproval","hasAnyGE","targetType","diseaseType"]]
#approvalsRusinaGE["therapeuticArea"] = approvalsRusinaGE.apply(lambda row: "oncology" if row.therapeuticArea == "Oncology" else "non-oncology", axis=1)

results = (
    
    # Rusina approvals
    approvalsRusinaIndirectGE.groupby(["yearApproval"]).count()[["drugId"]].reset_index().rename(columns={"drugId":"Approvals","yearApproval":"year"})
    
    # Rusina approvals supported by direct GE linking original target and original disease
    .merge(approvalsRusinaIndirectGE[(approvalsRusinaDirectGE.hasAnyGE==True)&(approvalsRusinaDirectGE.targetType=="Original")&(approvalsRusinaDirectGE.diseaseType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"Direct GE between original target and original disease","yearApproval":"year"}), on="year", how="left")
    
    # Rusina approvals supported by indirect GE linking original target and original disease
    .merge(approvalsRusinaIndirectGE[(approvalsRusinaIndirectGE.hasAnyGE==True)&(approvalsRusinaIndirectGE.targetType=="Original")&(approvalsRusinaIndirectGE.diseaseType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
           .rename(columns={"drugId":"Indirect GE between original target and original disease","yearApproval":"year"}), on="year", how="left")
    
#     # Rusina approvals supported by indirect GE linking original/interactor target and original/related disease
#     .merge(approvalsRusinaGE[(approvalsRusinaIndirectGE.hasAnyGE==True)].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
#            .rename(columns={"drugId":"Indirect GE between original/interacting target and original/related disease","yearApproval":"year"}), on="year", how="left")      
#     # Rusina approvals supported by indirect GE linking original target and original/related disease
#     .merge(approvalsRusinaIndirectGE[(approvalsRusinaIndirectGE.hasAnyGE==True)&(approvalsRusinaIndirectGE.targetType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
#            .rename(columns={"drugId":"Indirect GE between original target and original/related disease","yearApproval":"year"}), on="year", how="left")
#     # Rusina approvals supported by indirect GE linking original/interactor target and original disease
#     .merge(approvalsRusinaIndirectGE[(approvalsRusinaIndirectGE.hasAnyGE==True)&(approvalsRusinaIndirectGE.diseaseType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
#            .rename(columns={"drugId":"Indirect GE between original/interacting target and original disease","yearApproval":"year"}), on="year", how="left")    
    
#     # Rusina approvals supported by direct GE linking original/interactor target and original/related disease
#     .merge(approvalsRusinaGE[(approvalsRusinaDirectGE.hasAnyGE==True)].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
#            .rename(columns={"drugId":"Direct GE between original/interacting target and original/related disease","yearApproval":"year"}), on="year", how="left")      
    
#     # Rusina approvals supported by direct GE linking original target and original/related disease
#     .merge(approvalsRusinaIndirectGE[(approvalsRusinaDirectGE.hasAnyGE==True)&(approvalsRusinaDirectGE.targetType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
#            .rename(columns={"drugId":"Direct GE between original target and original/related disease","yearApproval":"year"}), on="year", how="left")
#     # Rusina approvals supported by direct GE linking original/interactor target and original disease
#     .merge(approvalsRusinaIndirectGE[(approvalsRusinaDirectGE.hasAnyGE==True)&(approvalsRusinaDirectGE.diseaseType=="Original")].groupby(["yearApproval"]).count()[["drugId"]].reset_index()
#            .rename(columns={"drugId":"Direct GE between original/interacting target and original disease","yearApproval":"year"}), on="year", how="left")    
)
#results["%automaticGE"] = results["automaticGE"]/results["automaticDrugId"]

results["%Direct GE between original target and original disease"] = results["Direct GE between original target and original disease"]/results["Approvals"]
results["%Indirect GE between original target and original disease"] = results["Indirect GE between original target and original disease"]/results["Approvals"]

# results["%Indirect GE between original/interacting target and original/related disease"] = results["Indirect GE between original/interacting target and original/related disease"]/results["Approvals"]
# results["%Indirect GE between original target and original/related disease"] = results["Indirect GE between original target and original/related disease"]/results["Approvals"]
# results["%Indirect GE between original/interacting target and original disease"] = results["Indirect GE between original/interacting target and original disease"]/results["Approvals"]

# results["%Direct GE between original/interacting target and original/related disease"] = results["Direct GE between original/interacting target and original/related disease"]/results["Approvals"]
# results["%Direct GE between original target and original/related disease"] = results["Direct GE between original target and original/related disease"]/results["Approvals"]
# results["%Direct GE between original/interacting target and original disease"] = results["Direct GE between original/interacting target and original disease"]/results["Approvals"]


ax = plt.figure().add_subplot()

# sns.lineplot(data=results, x="year", y="Indirect GE between original/interacting target and original disease", marker=".", markersize=10, color="orange", ax=ax, 
#              label="Indirect GE between original/interacting target and original disease")
# sns.lineplot(data=results, x="year", y="Indirect GE between original target and original/related disease", marker=".", markersize=10, color="purple", ax=ax, 
#              label="%Indirect GE between original/interacting target and original disease")
# sns.lineplot(data=results, x="year", y="%Indirect GE between original target and original disease", marker=".", markersize=10, color="b", ax=ax, 
#              label="%Indirect GE between original target and original disease")

# sns.lineplot(data=results, x="year", y="Direct GE between original/interacting target and original/related disease", marker=".", markersize=10, color="g", 
#              label="Direct GE between original/interacting target and original/related disease", ax=ax, linestyle="--")
# sns.lineplot(data=results, x="year", y="Direct GE between original/interacting target and original disease", marker=".", markersize=10, color="orange", ax=ax, linestyle="--", 
#              label="Direct GE between original/interacting target and original disease")
# sns.lineplot(data=results, x="year", y="Direct GE between original target and original/related disease", marker=".", markersize=10, color="purple", ax=ax, linestyle="--", 
#              label="Direct GE between original/interacting target and original disease")

sns.lineplot(data=results, x="year", y="%Direct GE between original target and original disease", marker=".", markersize=10, color=blue, ax=ax, linestyle="--", 
             label="%Direct GE between original target and original disease")
sns.lineplot(data=results, x="year", y="%Indirect GE between original target and original disease", marker=".", markersize=10, color=blue, ax=ax, linestyle="-", 
             label="%Indirect GE between original target and original disease")

ax.set_ylim(0,1)
ax.set_xlim(2013,2023)
ax.set_ylabel("% Approvals")
ax.set_xlabel("Approval year")
leg = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), frameon=0)
#plt.savefig("../results/23.02/plots/approvals.png", dpi=300, bbox_inches="tight")

Legends

In [ ]:
# timestamps legend
legend_elements = []
for source, color in {'Genetic association': '#577590',
 'Clinical': '#f94144',
 'Literature': '#90be6d',
 'Animal model': '#f9c74f',
 'Somatic mutations': '#43aa8b',
 'RNA expression': '#f8961e',
 'Affected pathway': '#f3722c',
 }.items():
    legend_elements.append(Patch(facecolor=color, edgecolor='k', label=source))

plt.figure()
fig, ax = plt.subplots()
ax.legend(
    handles=legend_elements,
    frameon=False,
    fontsize=plotTimeseries.font["size"],
    loc="center left",
    bbox_to_anchor=(1, 0.5),
    ncols=1,
    title="Category:"
)
plt.savefig("../results/23.06/plots/legends.png", dpi=300, bbox_inches="tight")

In [ ]:
# bells legend
plt.rcParams['legend.handlelength'] = 1.5
plt.rcParams['legend.handleheight'] = 1.51
legend_elements = []
for source in ["Human genetic data","RNA expression", "Animal model","Literature","Affected pathway",
               "Clinical phase I/II","Clinical phase III","Clinical phase IV"]:
    legend_elements.append(
    Line2D(
        [0],
        [0],
        color=plotTimeseries.getDatatypeToColour(by="label")[source],
        lw=5,
        label=source,
    )    
    )
plt.figure()
fig, ax = plt.subplots()
ax.legend(
    handles=legend_elements,
    frameon=False,
    fontsize=plotTimeseries.font["size"],
    loc="center left",
    bbox_to_anchor=(1, 0.5),
    ncols=4,
)
plt.savefig("../results/23.06/plots/legends.png", dpi=300, bbox_inches="tight")

In [ ]:
# therapy areas
plt.rcParams['legend.handlelength'] = .7
plt.rcParams['legend.handleheight'] = 1.51
#plt.rcParams['legend.numpoints'] = 1

tas = {idx: name for idx, name in plotTimeseries.getTherapeuticAreaToName().items() if name not in ('phenotype', 'injury, poisoning or other complication',"disorder of ear","hematologic disease","pregnancy or perinatal disease","psychiatric disorder")}
cmap_therapeuticAreas = dict(zip([tas[ta] for ta in sorted(tas, key=lambda x: tas[x])],
                                 list(sns.color_palette("tab20", len(tas)).as_hex())))
legend_elements = []
for source in sorted(cmap_therapeuticAreas):
    legend_elements.append(Patch(facecolor=cmap_therapeuticAreas[source], edgecolor='w', label=source.capitalize()))
plt.figure()
fig, ax = plt.subplots()
ax.legend(
    handles=legend_elements,
    frameon=False,
    fontsize=plotTimeseries.font["size"],
    loc="center left",
    bbox_to_anchor=(1, 0.5),
    ncols=2,
)
plt.savefig("../results/23.06/plots/legends.png", dpi=300, bbox_inches="tight")

In [ ]:
# correlations legend
plt.rcParams['legend.handlelength'] = .9
plt.rcParams['legend.handleheight'] = 1.51
legend_elements = []
for source in ["Affected pathway", "Animal model", "Clinical","Genetic association", "Literature","RNA expression","Somatic mutations"]:
    legend_elements.append(Patch(facecolor=plotTimeseries.getDatatypeToColour(by="label")[source], edgecolor='w', label=source))

plt.figure()
fig, ax = plt.subplots()
ax.legend(
    handles=legend_elements,
    frameon=False,
    fontsize=plotTimeseries.font["size"],
    loc="center left",
    bbox_to_anchor=(1, 0.5),
    ncols=1,
    title="Category:"
)
plt.savefig("../results/23.06/plots/legends.png", dpi=300, bbox_inches="tight")

Novels

In [ ]:
# novels heatmap
red = "#e40a36"
orange = "#ebb871"
green = "#44952a"

novels = pd.read_csv("/Users/mariaf/TargetEngine/results/23.06/novels/novels.csv").sort_values(["Therapeutic area","association"], ascending=[False, False]).set_index("association").T[::-1].T
repurposed = pd.read_csv("/Users/mariaf/TargetEngine/results/23.06/novels/repurposed.csv").sort_values(["Therapeutic area","association"], ascending=[False, False]).set_index("association").T[::-1].T
novels = pd.concat([repurposed,novels]).sort_values(["Therapeutic area","association"]).dropna(axis="columns", how="all")
#set1 = novels.head(50)#.dropna(axis="columns", how="all")
#set2 = novels.tail(50)#.dropna(axis="columns", how="all")

#for novels in (set1, set2):
if 1:
    drugs = novels[["Indication therapeutic area", "Drug"]]
    novels = (novels[["Therapeutic area"]+sorted([col for col in novels.columns if "novelty" in col])])
    novels = novels.rename(columns={col: col.replace(" novelty","") for col in novels.columns})

    data = novels.values.T

    cmap_datasources = LinearSegmentedColormap.from_list('cmap_datasources', ["white",'#2AAA69'], N=256)
    custom_colors = [(red, -1), (orange, 0), (green, 1)]
    cmap_target = ListedColormap([color for color, value in custom_colors], name='cmap_target')
    norm = plt.Normalize(min(value for color, value in custom_colors), max(value for color, value in custom_colors))
    tas = {idx: name for idx, name in plotTimeseries.getTherapeuticAreaToName().items() if name not in ('phenotype', 'injury, poisoning or other complication',"disorder of ear","hematologic disease","pregnancy or perinatal disease","psychiatric disorder")}
    cmap_therapeuticAreas = dict(zip([tas[ta] for ta in sorted(tas, key=lambda x: tas[x])],
                                    list(sns.color_palette("tab20", len(tas)).as_hex())))    
    cmap_therapeuticAreas[np.nan]="w"

    sns.set_style("white")
    fig, ax = plt.subplots(figsize=(30,25))

    cell_radius = 0.4
    square_size = 0.8  # Size of the square cells
    edge_color = 'gray'  # Color for the edge line

    for j in range(data.shape[1]):  
        for i in range(data.shape[0]):   
            if 0:
                pass                
            # if "Association score" in novels.columns[i]:
            #     square = Rectangle((i - square_size / 2, j - square_size / 2), square_size, square_size,
            #                        color=cmap_datasources(data[i, j]), edgecolor=edge_color, linewidth=0.5)
            #     ax.add_patch(square)
            #     if data[i, j] == 0:
            #         square.set_edgecolor(edge_color)
            #         square.set_linewidth(0.5) 
            
            if "Therapeutic area" in novels.columns[i]:  
                square = Rectangle((i , j - square_size / 2), .3, square_size,
                                color=cmap_therapeuticAreas[data[i, j]], edgecolor=edge_color, linewidth=0.5)
                ax.add_patch(square)
                if data[i, j] == 0:
                    square.set_edgecolor(edge_color)
                    square.set_linewidth(0.5)
            # elif "Drug" in novels.columns[i]:  
            #     ax.annotate(data[i,j], (i,j))
            else: # "novelty" in novels.columns[i]:
                circle = Circle((i, j), cell_radius, color=cmap_datasources(data[i, j]))
                ax.add_patch(circle)
                if (data[i, j] == 0) or (np.isnan(data[i, j])):
                    circle.set_edgecolor(edge_color)
                    circle.set_linewidth(0.2)                 

    ax.annotate("Drug for\nrepurposing", (i+3.2,j+1), fontsize=17, va="bottom", ha="center", annotation_clip=False)
    for idx, (ta, drug) in enumerate(drugs.values):
        if drug != drug:            
            txt = ""
        else:
            txt = drug.capitalize()
        if ta in ("congential disorder", "cancer or benign tumor","pancreas disease","endocrine system disease","immune system disease"):
            txt_colour = "w"
        else:
            txt_colour = "k"
        ax.annotate(txt, (i+3.2,idx), fontsize=16, color=txt_colour, va="center", ha="center", bbox=dict(boxstyle='round,pad=.2', facecolor=cmap_therapeuticAreas[ta]), annotation_clip=False)
                
    ax.set_aspect('equal')
    ax.set_frame_on(False)
    ax.set_xlim(-0.5, data.shape[0] - 0.5)  
    ax.set_ylim(-0.5, data.shape[1] - 0.5)
    ax.set_yticks(np.arange(data.shape[1]))  
    ax.set_yticklabels(novels.index)  
    ax.xaxis.tick_top()
    ax.set_xticks(np.arange(data.shape[0]))
    ax.set_xticklabels(novels.columns, rotation=50, ha="left")
    ax.xaxis.set_ticks_position('none')
    plt.savefig("../results/23.06/plots/novels.png", dpi=100, bbox_inches="tight")
    plt.show()


In [ ]:
# wordclouds
data = (spark.read.parquet("../results/23.06/novels/novels")
        .filter(~F.col("targetSymbol").isin(["CDCP1","TSPAN31","TMED3","NHIP","CASC20","DUSP5P1"]))
        .filter(~F.col("therapeuticAreaName").isin(["reproductive system or breast disease","integumentary system disease"]))    
        .filter((F.col("datasources").contains("europepmc"))|(F.col("datasources").contains("ot_genetics_portal")))  
        .withColumn("metric", F.col("meanNovelty"))
        #.groupby("targetSymbol").agg(F.max("metric").alias("metric"))
        .orderBy("metric", ascending=False)
        #.limit(10)
        .withColumn("therapeuticAreaName", 
                    F.when(F.col("therapeuticAreaName")=="cancer or benign tumor", "Oncological diseases")
                    .when(F.col("therapeuticAreaName")=="psychiatric disorder", "Neuronal diseases")
                    .when(F.col("therapeuticAreaName")=="musculoskeletal or connective tissue disease", "Musculoskeletal diseases")
                    .otherwise("Other diseases")
        )
 ).toPandas()

target2score = dict(zip(data['targetSymbol'], data['metric']))

therapeuticAreaNames = sorted(data.therapeuticAreaName.unique())

def fixed_color_func(word, **kwargs):
    return "rgb(0, 0, 139)"  # Dark Blue

# Loop through each therapy area and generate a word cloud
for i, therapeuticAreaName in enumerate(therapeuticAreaNames):
    if therapeuticAreaName=="Neuronal diseases": continue
    if therapeuticAreaName=="Oncological diseases": continue
    if therapeuticAreaName=="Other diseases": continue
    if therapeuticAreaName=="Musculoskeletal diseases": continue

    # Filter the dataframe for the current therapy area
    data_filtered = data[data['therapeuticAreaName'] == therapeuticAreaName]
    if len(data_filtered) == 0: continue
    
    # Create a dictionary of protein names and their corresponding scores
    target2score = dict(zip(data_filtered['targetSymbol'], data_filtered['metric']))
    
    print(therapeuticAreaName)
    # Generate the word cloud
    wordcloud = WordCloud(width=800, height=500, background_color='white', #color_func=blue_color_func,
                          font_path="/Library/Fonts/Roboto-Bold.ttf", 
                          colormap='Blues').generate_from_frequencies(target2score)
    
    # Plot the word cloud in the respective subplot
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')  # No axis for word cloud display
    plt.savefig("../results/23.06/plots/wordclouds/{}.png".format(therapeuticAreaName), dpi=300, bbox_inches="tight")

In [ ]:
pd.read_csv("/Users/mariaf/TargetEngine/results/24.09/ENCOREv6/ENCORE-COLO_final_GENE_annotated_duality.csv")
data[data.targetSymbol=="FAUP4"]

In [ ]:
# bar chart: targetsByTherapeuticAreaName
data = (spark.read.parquet("../results/23.06/novels/targetsByTherapeuticAreaName0.4")
 .withColumnRenamed("novelty","targets")
 .replace({"cancer or benign tumor":"Oncological diseases",
           'nervous system disease':"Neuronal diseases",
           'musculoskeletal or connective tissue disease':"Musculoskeletal diseases",
           "genetic, familial or congenital disease":"Genetic diseases",
           "respiratory or thoracic disease":"Respiratory diseases",
           'integumentary system disease':"Integumentary diseases",
           'immune system disease':"Immune diseases",
           'reproductive system or breast disease':"Reproductive diseases",
           'nutritional or metabolic disease':"Metabolic diseases",
           'disorder of visual system':"Visual diseases",
           'urinary system disease':"Urinary diseases",
           "infectious disease":"Infectious diseases",
           "pancreas disease":"Pancreas diseases",
           "gastrointestinal disease":"Gastrointestinal diseases",
           "cardiovascular disease":"Cardiovascular diseases",
           })
 .orderBy("targets", ascending=False)
  .limit(15)
 ).toPandas()


plt.figure(figsize=(5,5)) 
ax = sns.barplot(data=data, x="therapeuticAreaName", y="targets", color=plotTimeseries.getDatatypeToColour()["genetic_association"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, ha="right")
ax.set_xlabel("")
ax.set_ylabel("Unique targets\nwith novelty signal")
ax.set_title("Therapeutic area")
sns.despine()
plt.savefig("../results/23.06/plots/targetsByTherapeuticAreaName.png", dpi=300, bbox_inches="tight")

In [ ]:
# bar chart: targetsByBiotype
data = (spark.read.parquet("../results/23.06/novels/targetsByBiotype0.4")
 .orderBy("targets", ascending=False)
  .limit(15)
 ).toPandas()

plt.figure(figsize=(5,5)) 
ax = sns.barplot(data=data, x="targetBiotype", y="targets", color=plotTimeseries.getDatatypeToColour()["genetic_association"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, ha="right")
ax.set_xlabel("")
ax.set_ylabel("Unique targets\nwith novelty signal")
ax.set_title("Therapeutic area")
sns.despine()
#plt.savefig("../results/23.06/plots/targetsByTherapeuticAreaName.png", dpi=300, bbox_inches="tight")

In [ ]:
# bar chart: targetsByDatasource
data = (spark.read.parquet("../results/23.06/novels/targetsByDatasource0.4")
 .withColumnRenamed("novelty","targets")
 .orderBy("targets", ascending=False)
  .limit(15)
 ).toPandas()

data["datatypeId"] = data.apply(lambda row: plotTimeseries.getDatatypeForDatasource(output="dict")[row.datasourceId], axis=1)
data = data.replace(plotTimeseries.getDatasourceToName(output="dict"))

fig = plt.figure(figsize=(5,5)) 
#fig.set_facecolor('w')
ax = sns.barplot(data=data, x="datasourceId", y="targets", hue="datatypeId", palette=plotTimeseries.getDatatypeToColour(), legend=False)
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, ha="right")
ax.set_xlabel("")
ax.set_ylabel("Unique targets\nwith novelty signal")
ax.set_title("Novelty source")
sns.despine()
plt.savefig("../results/23.06/plots/targetsBySource.png", dpi=300, bbox_inches="tight")

In [ ]:
# bar chart: targetsByDatatype
data = (spark.read.parquet("../results/23.06/novels/targetsByDatatype0.4")
 .orderBy("targets", ascending=False)
  .limit(15)
 ).toPandas()

data = data.replace(plotTimeseries.getDatasourceToName(output="dict"))

fig = plt.figure(figsize=(5,5)) 
#fig.set_facecolor('w')
ax = sns.barplot(data=data, x="datatypeId", y="targets", hue="datatypeId", legend=False)
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, ha="right")
ax.set_xlabel("")
ax.set_ylabel("Unique targets\nwith novelty signal")
ax.set_title("Novelty source")
sns.despine()
#plt.savefig("../results/23.06/plots/targetsBySource.png", dpi=300, bbox_inches="tight")